In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import integrate

In [3]:
def squared_ratio_from_normal(normal):
    vertices_ucs = list(np.where(normal > 0)[0])
    normal_short = normal[vertices_ucs]
    factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])
    summands = np.prod(factors, axis=1, where=np.isfinite(factors))
    return np.sum(summands)**2

In [7]:
def f3(y, x, ssb_matrix):
    normal = x*ssb_matrix[:, 0] + y*ssb_matrix[:, 1] + (1-x-y)*ssb_matrix[:, 2]
    if len(np.unique(normal)) < len(normal):
        return 1  # upper bound
    return squared_ratio_from_normal(normal)

ssb_tr = np.array([[0, 1, 1], [-1, 0, 1], [-1, -1, 0]])
ssb_intr = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])

val, err = integrate.dblquad(f3, 0, 1, 0, lambda x: 1-x, args=(ssb_tr,))
print(val, 1-6*val, 6*err)
val, err = integrate.dblquad(f3, 0, 1, 0, lambda x: 1-x, args=(ssb_intr,))
print(val, 1-6*val, 6*err)

/tmp/ipykernel_43143/2759770424.py:4: RuntimeWarning: divide by zero encountered in divide
  factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])


0.1666666665048567 9.708598369684296e-10 8.107277773710648e-08
0.1257964123222241 0.24522152606665537 8.916985680924442e-08


In [5]:
def f4(z, y, x, ssb_matrix):
    normal = x*ssb_matrix[0, :] + y*ssb_matrix[1, :] + z*ssb_matrix[2, :] + (1-x-y-z)*ssb_matrix[3, :]
    if len(np.unique(normal)) < len(normal):
        return 1  # upper bound
    return squared_ratio_from_normal(normal)

m = 4  #number of alternative
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m))) # transitive preferences
# 4 cycle
ssb_intr1 = np.array([[0, 1, 1, -1], [-1, 0, 1, 1], [-1, -1, 0, 1], [1, -1, -1, 0]])
# 3 cycle preferred to the last option
ssb_intr2 = np.array([[0, 1, -1, 1], [-1, 0, 1, 1], [1, -1, 0, 1], [-1, -1, -1, 0]])
# 3 cycle not preferred to the last option
ssb_intr3 = np.array([[0, 1, 1, 1], [-1, 0, 1, -1], [-1, -1, 0, 1], [-1, 1, -1, 0]])


val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_tr,))
print(val, 1-18*val, 18*err)
val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr1,))
print(val, 1-18*val, 18*err)
val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr2,))
print(val, 1-18*val, 18*err)
val, err = integrate.tplquad(f4, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr3,))
print(val, 1-18*val, 18*err)

/tmp/ipykernel_43143/2759770424.py:4: RuntimeWarning: divide by zero encountered in divide
  factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])


0.055482959564882715 0.0013067278321111653 2.670579600648652e-07
0.048813214922008956 0.12136213140383878 2.68168699357802e-07
0.0528893484898806 0.04799172718214928 2.681968317509993e-07
0.05288934839985304 0.04799172880264524 2.6819133874096955e-07


In [ ]:
def f5(z, y, x, w, ssb_matrix):
    normal = w*ssb_matrix[0, :] +  x*ssb_matrix[1, :] + y*ssb_matrix[2, :] + z*ssb_matrix[3, :] + (1-x-y-z-w)*ssb_matrix[4, :]
#    if len(np.unique(normal)) < len(normal):
#        print("bound used")
#        return 1  # upper bound
    return squared_ratio_from_normal(normal)

m = 5  #number of alternative
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m))) # transitive preferences
# symmetric cycle
ssb_intr1 = np.array([[0, 1, 1, -1, -1], [-1, 0, 1, 1, -1], [-1, -1, 0, 1, 1], [1, -1, -1, 0, 1], [1, 1, -1, -1, 0]])
# 3 cycle preferred to the other options
ssb_intr2 = np.array([[0, 1, -1, 1, 1], [-1, 0, 1, 1, 1], [1, -1, 0, 1, 1], [-1, -1, -1, 0, 1], [-1, -1, -1, -1, 0]])
# 3 cycle in the middle
ssb_intr3 = np.array([[0, 1, 1, 1, 1], [-1, 0, 1, -1, 1], [-1, -1, 0, 1, 1], [-1, 1, -1, 0, 1], [-1, -1, -1, -1, 0]])


val, err = integrate.nquad(f5, [lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_tr])
print(val, 1-72*val, 72*err)
val, err = integrate.nquad(f5, [lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_intr1])
print(val, 1-72*val, 72*err)
val, err = integrate.nquad(f5, [lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_intr2])
print(val, 1-72*val, 72*err)
val, err = integrate.nquad(f5, [lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_intr3])
print(val, 1-72*val, 72*err)
# Computation took 134min

/tmp/ipykernel_9538/2759770424.py:4: RuntimeWarning: divide by zero encountered in divide
  factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])
/home/johanna/miniconda3/envs/py312-MA/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


0.01385671084635187 0.002316819062665343 5.527654480809081e-05
0.010435501758815692 0.24864387336527016 1.0727949562815917e-06
0.013619070592128052 0.01942691736678026 1.071437161784159e-06


/home/johanna/miniconda3/envs/py312-MA/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/johanna/miniconda3/envs/py312-MA/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


0.013610010619357711 0.02007923540624479 0.0002183789200921795


In [ ]:
def f6(z, y, x, w, v, ssb_matrix):
    normal = v*ssb_matrix[0, :] + w*ssb_matrix[1, :] +  x*ssb_matrix[2, :] + y*ssb_matrix[3, :] + z*ssb_matrix[4, :] + (1-x-y-z-w-v)*ssb_matrix[5, :]
#    if len(np.unique(normal)) < len(normal):
#        print("bound used")
#        return 1  # upper bound
    return squared_ratio_from_normal(normal)

m = 6  #number of alternative
ssb_tr = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m))) # transitive preferences
# symmetric cycle
# ssb_intr1 = np.array([[0, 1, 1, -1, -1], [-1, 0, 1, 1, -1], [-1, -1, 0, 1, 1], [1, -1, -1, 0, 1], [1, 1, -1, -1, 0]])
# 3 cycle preferred to the other options
ssb_intr2 = np.array([[0, 1, -1, 1, 1], [-1, 0, 1, 1, 1], [1, -1, 0, 1, 1], [-1, -1, -1, 0, 1], [-1, -1, -1, -1, 0]])


val, err = integrate.nquad(f6, [lambda w, x, y, z, _: [0, 1-w-x-y-z], lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_tr])
print(val, 1-360*val, 360*err)
val, err = integrate.nquad(f6, [lambda w, x, y, z, _: [0, 1-w-x-y-z], lambda x, y, z, _: [0, 1-x-y-z], lambda y, z, _: [0, 1-y-z], lambda z, _: [0, 1-z], [0, 1]], args = [ssb_intr2])
print(val, 1-360*val, 360*err)
# Stopped after 200min

/tmp/ipykernel_9538/2759770424.py:4: RuntimeWarning: divide by zero encountered in divide
  factors = normal_short[:, None]/(normal_short[:, None] - normal[None, :])
/home/johanna/miniconda3/envs/py312-MA/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/home/johanna/miniconda3/envs/py312-MA/lib/python3.12/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(

KeyboardInterrupt: 